# fct_operator_hourly_summary

In [5]:
import datetime
import sys

import _operator_grain_route_dir_visuals as _report_operator_visuals
import _operator_grain_scheduled_service
import _sql_query
import google.auth
import pandas as pd
from google.cloud import bigquery
from IPython.display import HTML, Image, Markdown, display, display_html
from loguru import logger
from omegaconf import OmegaConf
from shared_utils import gtfs_utils_v2, portfolio_utils, publish_utils, rt_dates
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SEGMENT_GCS
from functools import cache

import altair as alt
import geopandas as gpd
from shapely import wkt

from omegaconf import OmegaConf
readable_dict = OmegaConf.load("new_readable.yml")

In [6]:
from calitp_data_analysis.gcs_pandas import GCSPandas

In [7]:
@cache
def gcs_pandas():
    return GCSPandas()

In [8]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [9]:
analysis_name = "City and County of San Francisco"

In [10]:
url = "gs://calitp-analytics-data/data-analyses/gtfs_digest/processed/fct_operator_hourly_summary_2025_12.parquet"

In [12]:
df = gcs_pandas().read_parquet(url)

In [ ]:
df.sample(3)

In [13]:
df["Departure Hour"].unique()

<IntegerArray>
[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
Length: 32, dtype: Int64

In [14]:
sf_only = df.loc[df["Analysis Name"] == analysis_name]

In [15]:
sf_only.loc[sf_only["Departure Hour"]> 24]

,Analysis Name,Date,Day Type,Departure Hour,N Trips
6996,City and County of San Francisco,12-2024,Saturday,25,53
6997,City and County of San Francisco,12-2024,Saturday,26,53
6998,City and County of San Francisco,12-2024,Saturday,27,53
6999,City and County of San Francisco,12-2024,Saturday,28,52
7000,City and County of San Francisco,12-2024,Saturday,29,36
7001,City and County of San Francisco,12-2024,Saturday,30,5
7023,City and County of San Francisco,12-2024,Sunday,25,53
7024,City and County of San Francisco,12-2024,Sunday,26,53
7025,City and County of San Francisco,12-2024,Sunday,27,53
7026,City and County of San Francisco,12-2024,Sunday,28,52


In [ ]:
df["Date"].unique()

In [ ]:
def create_hourly_summary(df: pd.DataFrame, day_type: str):
    
    chart_dict = readable_dict.hourly_summary
    df2 = df.loc[df["Day Type"] == "Saturday"]
    df2["Date"] = df["Date"].astype(str)
    
    date_list = list(df2["Date"].unique())
    
    date_dropdown = alt.binding_select(
        options=date_list,
        name="Dates: ",
    )
    xcol_param = alt.selection_point(
        fields=["Date"], value=date_list[0], bind=date_dropdown
    )

    chart = (
        (
            alt.Chart(df2)
            .mark_line(size=3)
            .encode(
                x=alt.X(
                    "Departure Hour",
                    title="Departure Hour",
                    axis=alt.Axis(
                        labelAngle=-45,
                    ),
                ),
                y=alt.Y(
                    "N Trips",
                    title="N Trips",
                ),
            )
        )
        .add_params(xcol_param)
        .transform_filter(xcol_param)
    )
    
    bg = _operator_grain_scheduled_service.create_bg_service_chart()
    
    chart = (chart + bg).properties(
    resolve=alt.Resolve(
        scale=alt.LegendResolveMap(color=alt.ResolveMode("independent"))
    )
)
    chart = _report_operator_visuals.configure_chart(
    chart,
    width=400,
    height=250,
    title=f"{chart_dict.title} {day_type}",
    subtitle=chart_dict.subtitle)
    
    return chart

In [ ]:
create_hourly_summary(sf_only, "Saturday")

In [ ]:
create_hourly_summary(sf_only, "Sunday")

In [ ]:
create_hourly_summary(sf_only, "Weekday")